$|\dot{\psi}\rangle = -iH|\psi\rangle$

For each edge, $(i,j)$, we include a term: 
$$
H_{ij}=I\otimes\cdots\otimes I\otimes\sigma_z\otimes I\otimes\cdots\otimes I\otimes\sigma_z\otimes I\otimes\cdots\otimes I
$$ 
in the Hamiltonian, where the two $\sigma_z$ appear at the $i$ and $j$ locations in the tensor product. Note that for a state:

$|\psi\rangle = |q_0\rangle\otimes\cdots |q_{N-1}\rangle$, we have 
$$
H_{ij}|\psi\rangle=|q_0\rangle\otimes |q_{i-1}\rangle\otimes\sigma_z |q_i\rangle\otimes|q_{i+1}\rangle\cdots|q_{j-1}\rangle\otimes\sigma_z|q_j\rangle\otimes |q_{j+1}\rangle\otimes |q_{N-1}\rangle
$$ 

And so $\langle \psi|H_{ij}|\psi\rangle = \langle q_i|\sigma_z|q_i\rangle\langle q_j|\sigma_z|q_j\rangle=(1-2q_i)(1-2q_j)$, which is $1$ if $i$ and $j$ are in the same state, and $-1$ if they are in opposite states. If we are talking about colorings, then for each edge violation we will get a +1, and for each non-violation we will get a -1. 

Therefore, if $H=\sum_{e\in E(G)}H_e$, then the energy of a coloring is the number of violations minus non-violations. If we use $(H+mI)/2$ instead, the energy will be equal to the number of bad edges. If we use $(H-mI)/2$ it will be negative the number of good edges. 

Rather than working with $H$ directly, we can get the same thing by modifying $H_{ij}$ to either $(H_{ij}+I)/2)$ or $H_{ij}-I)/2$. However, I don't think these can be represented as Kronecker products. We would need to use a sum of Kronecker products: $(\sigma_z\otimes \sigma_z+I\otimes I)/2$, for example.

In [295]:
import torch
import numpy as np

# We will have energy represent the number of edge violations for a coloring

# # Petersen graph adjacency matrix
# A = torch.tensor([
#     [0,1,0,0,1,1,0,0,0,0],
#     [1,0,1,0,0,0,1,0,0,0],
#     [0,1,0,1,0,0,0,1,0,0],
#     [0,0,1,0,1,0,0,0,1,0],
#     [1,0,0,1,0,0,0,0,0,1],
#     [1,0,0,0,0,0,0,1,1,0],
#     [0,1,0,0,0,0,0,0,1,1],
#     [0,0,1,0,0,1,0,0,0,1],
#     [0,0,0,1,0,1,1,0,0,0],
#     [0,0,0,0,1,0,1,1,0,0]],dtype=torch.int8)

# 5 cycle
A = torch.tensor([
    [0,1,0,0,1],
    [1,0,1,0,0],
    [0,1,0,1,0],
    [0,0,1,0,1],
    [1,0,0,1,0]],dtype=torch.int8)


N = A.shape[0]
M = torch.sum(A)//2

# qubit = [torch.tensor([0, 1]) for _ in range(10)]  # Create a list of qubits
# states = torch.cartesian_prod(*qubit)  # Create the cartesian product of the list

pauli_z = torch.tensor([[1, 0], [0, -1]], dtype=torch.int8) # Need to change for more than 2 colors...
print(pauli_z)
print(torch.kron(pauli_z,pauli_z))

H = torch.zeros(2**N,2**N,dtype=torch.int8) # Hamiltonian
for i in range(N):
    for j in range(i):
        if A[i, j] == 1:

            H_ij=torch.kron(torch.kron(torch.kron(torch.kron(
                torch.eye(2**j, dtype=torch.int8),
                pauli_z),
                torch.eye(2**(i-j-1), dtype=torch.int8)),
                pauli_z),
                torch.eye(2**(N-i-1), dtype=torch.int8))
            H += H_ij


up_down = torch.eye(2,dtype=torch.int8)
x = (torch.rand(N)<0.5).to(torch.int8)
print("Coloring: ", x)

coloring = torch.stack([up_down[b] for b in x])

state=torch.tensor(1,dtype=torch.int8)
for b in x:
    state = torch.kron(state,up_down[b])

x=x.unsqueeze(0)
state=state.unsqueeze(0)

bad_edges = torch.sum(A*(coloring@coloring.t()))//2
good_edges = M-bad_edges

print("# bad edges: ",bad_edges.item())
print("# good edges: ",good_edges.item())

energy = state@H@state.t()
print("Energy: ",energy.item())

# Check if the energy is correct
correct = energy == bad_edges - good_edges
print("Working: ", correct.item())

# print(torch.trace(torch.abs(H)))




tensor([[ 1,  0],
        [ 0, -1]], dtype=torch.int8)
tensor([[ 1,  0,  0,  0],
        [ 0, -1,  0,  0],
        [ 0,  0, -1,  0],
        [ 0,  0,  0,  1]], dtype=torch.int8)
Coloring:  tensor([1, 1, 0, 0, 1], dtype=torch.int8)
# bad edges:  3
# good edges:  2
Energy:  1
Working:  True


Now let's try adjusting the Hamiltonian...

In [109]:
H1 = (H+M)//2
H2 = (H-M)//2

energy1 = state@H1@state.t()
print("Energy representing bad edges: ",energy1.item())

energy2 = state@H2@state.t()
print("Energy representing (negative) good edges: ",energy2.item())


TypeError: unsupported operand type(s) for +: 'function' and 'Tensor'

Now if we want to use more than two colors, things become slightly tricky. We can define $H_{ij}$ as a sum of tensor products. Ignoring all of the factors of the identity, we are essentially looking for a matrix, $M=\sum_i K_i\otimes J_i$, having the property that for each pair of verticies, $\langle ab|M|ab\rangle=$ takes one value if $a=b$ and a different value if $a\neq b$. We can expand this expression as: 

$$
\sum_{i=1}^n \langle a|K_i|a\rangle\langle b|J_i|b\rangle
$$

Let $K_{ij}=K_{i,jj}$ and $J_{ij}=J_{i,jj}$. Then $K$ and $J$ are $n\times c$ matrices and $K^TJ$ should be a matrix that has one value on the diagonal and a different value on the off-diagonal. The simplest way to do this is let $J=K$ and have all column in $K$ have the same length, and have the angles (or dot products) between columns of $K$ be equal. In the original examples with two colors, I think that all entries in $K$ are $\pm 1$. Based on all of this, I think the natural generalization of the Pauli z matrix, would be a set of matrices that are basically cxc identity matrices with a single sign flip on the diagonal. Call this $\sigma_1,\ldots,\sigma_c$. Then $\sum_i \sigma_i\otimes \sigma_i$ gives the appropriate generalization of $H_{ij}$, and by adding/subtracting the identity matrix, we can get appropriate generalization of $H1$ and $H2$. One way of looking at this is that we are penalizing each possible same color separately. 

In [516]:
# We will have energy represent the number of edge violations for a coloring

# Petersen graph adjacency matrix
A = torch.tensor([
    [0,1,0,0,1,1,0,0,0,0],
    [1,0,1,0,0,0,1,0,0,0],
    [0,1,0,1,0,0,0,1,0,0],
    [0,0,1,0,1,0,0,0,1,0],
    [1,0,0,1,0,0,0,0,0,1],
    [1,0,0,0,0,0,0,1,1,0],
    [0,1,0,0,0,0,0,0,1,1],
    [0,0,1,0,0,1,0,0,0,1],
    [0,0,0,1,0,1,1,0,0,0],
    [0,0,0,0,1,0,1,1,0,0]],dtype=torch.int8)

# # 5 cycle
# A = torch.tensor([
#     [0,1,0,0,1],
#     [1,0,1,0,0],
#     [0,1,0,1,0],
#     [0,0,1,0,1],
#     [1,0,0,1,0]],dtype=torch.int8)

N = A.shape[0]
M = torch.sum(A)//2
K = 2 # Number of colors

# Create equivalent of Pauli Z
rho = [torch.zeros((K,K),dtype=torch.int8) for c in range(K)]
# Again, copilot guessed this loop!
##################################
for c in range(K):
    rho[c][c,c] = 1
##################################

H = torch.zeros(K**N,K**N,dtype=torch.int8) # Hamiltonian
for i in range(N):
    for j in range(i):
        if A[i, j] == 1:
            for c in range(K):
                H_c=torch.kron(torch.kron(torch.kron(torch.kron(
                    torch.eye(K**j, dtype=torch.int8),
                    rho[c]),
                    torch.eye(K**(i-j-1), dtype=torch.int8)),
                    rho[c]),
                    torch.eye(K**(N-i-1), dtype=torch.int8))
                H += H_c


up_down = torch.eye(K,dtype=torch.int8)
x = torch.randint(0, K, (N,))
# x = torch.tensor([0,1,0,1,0],dtype=torch.int8)
print("Coloring: ", x)

coloring = torch.stack([up_down[b] for b in x])

state=torch.tensor(1,dtype=torch.int8)
for b in x:
    state = torch.kron(state,up_down[b])

x=x.unsqueeze(0)
state=state.unsqueeze(0)

bad_edges = torch.sum(A*(coloring@coloring.t()))//2
good_edges = M-bad_edges

print("# bad edges: ",bad_edges.item())
# print("# good edges: ",good_edges.item())

energy = state@H@state.t()
print("Energy: ",energy.item())

# Check if the energy is correct
correct = energy == bad_edges
print("Working: ", correct.item())

Coloring:  tensor([0, 0, 0, 1, 1, 1, 1, 0, 1, 0])
# bad edges:  8
Energy:  8
Working:  True


In [299]:
state = torch.ones((K**N,1),dtype=torch.complex64)
state = state/torch.norm(state)
H_final = H.clone().to(torch.complex64)


pauli_x = (2.0/K)*torch.ones((K,K),dtype=torch.complex64) - torch.eye(K,dtype=torch.complex64)

H_init = torch.zeros((K**N,K**N),dtype=torch.complex64)
for i in range(N):
    H_init -= torch.kron(torch.kron(torch.eye(K**i,dtype=torch.complex64),pauli_x),torch.eye(K**(N-i-1),dtype=torch.complex64))
print(H_init.to(torch.float64))
n_steps = 5000
dt = 20/n_steps # Needs to be bigger than 1/n_steps in order to be adiabatic

energy = torch.conj(state.t())@H_init@state

print("Energy: ",energy.item())

def H(t):
    return (1-t)*H_init + t*H_final

for itr,t in enumerate(torch.linspace(0,1,n_steps)):
    state += -1j*dt*H(t)@state
    state = state/torch.norm(state)
    energy = torch.conj(state.t())@H(t)@state
    if (itr+1)%(n_steps//10)==0:
        print("Energy: ",energy.item())


tensor([[ 0., -1., -1.,  ...,  0.,  0.,  0.],
        [-1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [-1.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0., -1.],
        [ 0.,  0.,  0.,  ...,  0.,  0., -1.],
        [ 0.,  0.,  0.,  ..., -1., -1.,  0.]], dtype=torch.float64)
Energy:  (-10+0j)
Energy:  (-8.263263702392578-1.4784745872020721e-08j)
Energy:  (-6.5490875244140625+2.3283064365386963e-09j)
Energy:  (-4.872203826904297-1.0943040251731873e-08j)
Energy:  (-3.2497196197509766+1.0011717677116394e-08j)
Energy:  (-1.7066035270690918+1.1175870895385742e-08j)
Energy:  (-0.2803734540939331-5.587935447692871e-09j)
Energy:  (0.9849222898483276+4.6566128730773926e-09j)
Energy:  (2.0211472511291504-6.984919309616089e-09j)
Energy:  (2.7244138717651367-8.475035429000854e-08j)
Energy:  (3.058256149291992-2.3096877299622065e-08j)


Okay, so this is actually working pretty well. But the matrices are giantic! So, my question is whether there is any way to do the same thing in a lower dimensional space. I can think of two potential approaches. First, is to start with the current approach and try to project it onto a subspace that still allows the same dynamics. The other is to try to come up with a different formulation that looks similar. 

For the first method, we would be looking for a projection matrix, $P$, so the $|\xi\rangle = P|\psi\rangle$, would be our new variable. Clearly, we have 
$$
|\dot{\xi}\rangle = P|\dot{\psi}\rangle=-iPH|\psi\rangle
$$

For the other approach...

Here are a couple ideas. First, let's put a constraint that insists that if one block is non-zero so is another. In fact, the constraint I'm think of is that the sums of all the blocks should be equal. So, if one block has element $(a_1,\ldots, a_K)$ and another has $(b1,\ldots,b_K)$, then we can use the penalty $([\sum_i a_i]-[\sum_i b_i])^2$, which is quadratic and corresponds to the matrix $[[1,-1],[-1,1]]\otimes I_K$. Now that this is clearly symmetric and PSD. The idea would be to enforce this for all pairs of vertices (or at minimum for all edges on a tree). The idea is to create a symmetric PSD matrix that for every possible edge (either in the complete graph or the actual graph we're working with or a cycle...), includes the quadratic says that the vertex color sums are equal. So, the matrix has all non-negative eigenvalues, and the only way to get zero is to have all vertex color vectors have the same sum, and so color vectors with that property are ground states. We can no additionally enforce different edges to have significantly different states by including terms of dot products for adjacent verticies.  

The general goal is to come up with some representation of colorings such that an optimal coloring corresponds to a minimum eigenvector for a Hermitian matrix. If we consider the case of $K$ colors and a single edge we note a challenge. A Hermitian matrix expressible as a sum of orthogonal projections. Therefore, minimizing the $x^THx$ means minimizing the weights squares of $x^Tp$, for the various vectors $p$ onto which we are projecting. We want the different $p$ to represent good and bad states; i.e. colorings that either assign the same color or different colors to the two vertices. The problem is that there are at least $K^2$ colorings and only $2K$ dimensions. So, we need to create a lot of linear dependence between the colorings. That is, whatever representation we use, we want all of the bad colorings to form a subspace and all of the good colorings to form an orthogonal subsapce. Even with $K=2$, this is a challenge. It does not work to take the good colorings to be $(1,0,0,1)$ and $(0,1,1,0)$ and the bad to be $(1,0,1,0)$ and $(0,1,0,1)$, since the induced subspaces would not be disjoint, let alone orthogonal (both contain $(1,1,1,1)$). Really, all that is essential is that the good states form a subspace that do not include any bad states. That we, we can project out the good subspace, so the vectors will evaluate to zero if and only if they are good. My idea is that perhaps we can use complex rotations to allow more possible good states without increasing the actual dimension of their subspace.

